# Continuous Control

---


Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

## 0. Learning Algorithm

---

To solve Unity reacher problem, I choose [DDPG algorithm](https://arxiv.org/pdf/1509.02971.pdf)(Lillicrap et al., CONTINUOUS CONTROL WITH DEEP REINFORCEMENT LEARNING).

**Algorithm**

To make DQN agent deal with continuos action space, the authors suggest extension of actor-critic model called DDPG. The DDPG agent is consists of 2 kinds of networks. Actor network is responsible for the policy function approximator. Critic Network is reponsible for Q-value function approximator. 

Just like DQN, DDPG also uses replay buffer which stores old experiences and samples a small batch of tuples to remove correlations in consecutive observations. They also use target network used in DQN, but modified it to use soft target updates.

And they add Ornstein-Uhlenbeck Noise to the action produced by actor network for encouraging exploration. Lastly, they used Adam optimizer for learning the nueral network paramters.

<br>
<figure>
  <img src = "./ddpg_algorithm.png" width = 80% style = "border: thin silver solid; padding: 10px">
      <figcaption style = "text-align: center; font-style: italic">Fig 1. - DDPG Algorithm.</figcaption>
</figure> 
<br>


**Hyperparamters**

All these hyperparamters except buffer size and batch size are from the paper's experiment details. Buffer size and batch size are much smaller since the task is more simple.

```
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 128        # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR_ACTOR = 1e-4         # learning rate of the actor 
LR_CRITIC = 1e-3        # learning rate of the critic
WEIGHT_DECAY = 1e-2     # L2 weight decay
SIGMA = 0.2             # Paramter for OU Process
THETA = 0.15            # Paramter for OU Process

```

**Model Architecture**

Critic gets states and actions as input and ouputs the action-value. The paper states that the actions were not included until the 2nd hidden layer of Q. So In this Implementatiom, actions are merged into the hidden layer between the 1st and 2nd one.

## 1.Implementation


- Replay Buffer
- Actor and Critic Network
- OUNoise
- Agent

---

In [2]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import torch.nn.init as I

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

import numpy as np

### 1.Replay Buffer

Reinforcement learning is unstable when a nonlinear function approximator is used to represent action-value function, because the sequence of experiencs can be highly correlated. DDPG Agent stores the experience at each time step. Then by sampling from the buffer at random, It can prevent action values from oscillating or diverging.

In [3]:
from collections import deque, namedtuple
import random

class ReplayBuffer():
    def __init__(self, buf_size, batch_size, seed):
        """
        Params
        ----------
        buf_size (int): size of memory
        batch_size (int): number of samples to be sampled
        seed (int): random seed
        """
        # When the replay buffer was full, the oldest sample needs to be discarded.
        # So deque is suitalbe data structure. 
        self.memory = deque(maxlen=buf_size)
        self.seed = random.seed(seed)
        self.batch_size = batch_size
        self.experience = namedtuple('Trajectory', field_names=["state", "action", "reward", "next_state", "done"])
        
    def __len__(self):
        """
        Return the size of memory
        """
        
        return len(self.memory)
    def add(self, state, action, reward, next_state, done):
        """
        Add the agent's experiences at eacy time to the memory
        """
        # Instantiate new experience with custom nemaedTuple
        e = self.experience(state, action, reward, next_state, done)
        # Add the tuple to the memory
        self.memory.append(e)
        
    def sample(self):
        """
        Draw a sample.
        Since the sample data is used by pytorch model, It needs to be converted to a torch Tensor.
        
        Returns
        -------
        A tuple of torch tensor. Each tenosr's outermost dimension is batch_size.
        """
        # list of sampled experience namedtuple of size of self.batch_size
        experiences = random.sample(self.memory, k=self.batch_size)
        
        # states : [batch_size, state.shape]
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        # dones is needed to calculated the Q-value. At terminal state(dones=1), the Q-value should be just latest rewards.
        # Convert it to np.uint8
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        
        return (states, actions, rewards, next_states, dones)
        

### 2.Actor and Critic Networks

According to the papar, The authors initialized the final layer weights and biases of both the actor and critic from a uniform distribution $ [−3×10^−3, 3×10^−3] $ and $[3×10^−4, 3×10^−4]$. This was to ensure the initial outputs for the policy and value estimates were near zero. The other layers were initialized from uniform distributions $ [− \sqrt{f} , \sqrt{f} ] $ where f is the fan-in of the layer.

Since eveny entry in the action vector should be a number between -1 and 1, The activation function is **tanh**.
Other hidden layers is activated by **relu**. 

In [4]:
class Actor(nn.Module):
    """
    Policy Network : state -> specific action (Not Probability distribution of Actions)
    """
    def __init__(self, state_size, action_size, seed, fc1_units=64, fc2_units=128):
        """
        Initialization
        
        Params
        -------
        state_size : Vector Observation space size(per agent) : 33
        action_size : Vector Action space size(per agent) : 4
        seed : seed
        fc1_units : first hidden layer 
        fc2_units : seccond hidden layer
        """
        super(Actor, self).__init__()
        self.seed = torch.manual_seed(seed)
        
        self.fc1 = nn.Linear(in_features=state_size, out_features=fc1_units)
        self.fc2 = nn.Linear(in_features=fc1_units, out_features=fc2_units)
        #self.fc3 = nn.Linear(in_features=self.hidden2, out_features=self.hidden1)
        self.fc3 = nn.Linear(in_features=fc2_units, out_features=action_size)
        
        self.reset_parameters()
        
    def get_fan_in(self, layer):
        """
        Get the fan-in in each layer.
        """
        fan_in = 1/np.sqrt(layer.in_features)
        return -fan_in, fan_in
        
    def reset_parameters(self):
        """
        Initialize weights and bais in each layer
        """
        I.uniform_(self.fc1.weight, *self.get_fan_in(self.fc1))
        I.uniform_(self.fc2.weight, *self.get_fan_in(self.fc2))
        I.uniform_(self.fc3.weight, -3*1e-3, 3*1e+3)
            
    def forward(self, state):
        
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        # Every entry in action vector : [-1, 1]
        actions = F.tanh(self.fc3(x))
        
        return actions

In [5]:
class Critic(nn.Module):
    def __init__(self, state_size, action_size, seed, fc1_units=64, fc2_units=128):
        super(Critic, self).__init__()
        self.seed = torch.manual_seed(seed)
        
        # hidden layer for state pathway
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc_merged = nn.Linear(fc1_units+action_size, fc2_units) 
        self.fc2 = nn.Linear(fc2_units, fc1_units)
        self.fc3 = nn.Linear(fc1_units, action_size)
        
        # Initialize Weights and Biases
        self.reset_parameters()
        
    def reset_parameters(self):
        I.uniform_(self.fc1.weight, *self.get_fan_in(self.fc1))
        I.uniform_(self.fc_merged.weight, *self.get_fan_in(self.fc_merged))
        I.uniform_(self.fc2.weight, *self.get_fan_in(self.fc2))
        I.uniform_(self.fc3.weight, -3*1e-3, 3*1e+3)
        
    def get_fan_in(self, layer):
        fan_in = 1/np.sqrt(layer.in_features)
        return -fan_in, fan_in
    
    def forward(self, state, action):
        
        x = F.relu(self.fc1(state))
        # state : [batch_size, state_size], action : [batch_size, action_size]
        # merged : [batch_size, state_size + action_size]
        x = torch.cat((x, action), dim=1)
        x = F.relu(self.fc_merged(x))
        x = F.relu(self.fc2(x))
        q = self.fc3(x)
        return q

### 3.OUNoise

To encourage agnet do exploration at initial step, add noise from Ornstein–Uhlenbeck noise process to the specific action produced by the actor(policy) network. The variation of the noise process decreases as time goes by. Therefore it can lead to reducing the exploration as the agent train. Also 2 consecutive samples are temporally correlated. This will ensure that 2 consecutive actions are not different widly. The authors use theta=0.15 sigma=0.2 for this noise process.

In [6]:
import copy 

class OUNoise:
    """
    Ornstein–Uhlenbeck noise
    """
    def __init__(self, size, seed, mu=0., theta=0.15, sigma=0.2):
        """
        Intialization
        
        Params
        ------
        size : number of noise. It should be action_size = size
        seed : random seed
        mu : mean, defalut : 0 to reduce the noise over time.
        theta : hyper paramter
        sigam : hyper paramter
        """
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.seed = random.seed(seed)
        self.reset()
    def reset(self):
        """
        Reset the internal state to mu
        """
        self.state = copy.copy(self.mu)
    def sample(self):
        """
        Process the internal state.
        The Wiener process states are sampled by random.random()
        
        Returns
        -----
        self.state
        """
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma*np.array([random.random() for i in range(len(x))])
        self.state = x+dx
        return self.state

### 4.Agent

In [7]:
# Hyper parameters
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 128        # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR_ACTOR = 1e-4         # learning rate of the actor 
LR_CRITIC = 1e-3        # learning rate of the critic
WEIGHT_DECAY = 1e-2     # L2 weight decay

class DDPGAgent():
    """
    RL Agent whose actor and critic networks 
    """
    
    def __init__(self, state_size, action_size, seed):
        """
        Initialize an Agent object.
        
        Params
        -------
        state_size : 
        
        """
        self.state_size = state_size
        self.action_size = action_size
        self.seed = seed
        
        # Actor networks
        self.actor_local = Actor(state_size, action_size, seed).to(device)
        self.actor_target = Actor(state_size, action_size, seed).to(device)
        self.actor_optimizer = optim.Adam(self.actor_local.parameters(), lr=LR_ACTOR)
        # Critic networks
        self.critic_local = Critic(state_size, action_size, seed).to(device)
        self.critic_target = Critic(state_size, action_size, seed).to(device)
        self.critic_optimizer = optim.Adam(self.critic_local.parameters(), lr=LR_CRITIC, weight_decay=WEIGHT_DECAY)
        # Replay buffer
        self.memory = ReplayBuffer(BUFFER_SIZE, BATCH_SIZE, seed)
        # OUNoise
        self.noise = OUNoise(size=action_size, seed=self.seed)
    def reset(self):
        """
        Reset the OUNoise state.
        """
        self.noise.reset()
        
    def act(self, state, add_noise=True):
        """
        Returns actions for state based on current policy.
        Params
        -----
        state : numpy array [1xstate_size]
        add_noise : boolean , default: True
        Returns
        -----
        action : numpy array ,shape:[1 x action_size]
        """
        # Convert state into torch tensor
        state = torch.from_numpy(state).float().to(device)
        # Set evaluation mode
        self.actor_local.eval()
        # Disable gradient calculation
        with torch.no_grad():
            action = self.actor_local(state).cpu().data.numpy()
        self.actor_local.train()
        
        if add_noise:
            action += self.noise.sample()
            
        return np.clip(action, -1, 1)
    
    def step(self, state, action, reward, next_state, done):
        """
        Store experience in replay buffer
        """
        self.memory.add(state, action, reward, next_state, done)
        
        if len(self.memory)>BATCH_SIZE:
            # Get minibatch experiences
            experiences = self.memory.sample()
            self.learn(experiences, GAMMA)
    
    def learn(self, experiences, gamma):
        """
        Update actor and critic network
        
        Params
        -----
        experiences (Tuple[torch.Tensor])
        gamma (float)
        """
        # Each of them is batch sized torch Tenosr
        states, actions, rewards, next_states, dones = experiences
        
        # Update critic
        # 1. Get next actions
        next_actions = self.actor_target(next_states)
        # 2. Get target value from the target network 
        y = rewards + gamma*self.critic_target(states, next_actions)*(1-dones)
        # 3. Critic objective function
        critic_loss = F.mse_loss(self.critic_local(states, actions), y)
        # 4. Minimize
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()
        
        # Update actor
        # 1. Get predicted action
        pred_actions = self.actor_local(states)
        # 2. Actor objective
        actor_loss = -self.critic_local(states, pred_actions).mean()
        # 3. Minimize loss
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        
        # Update target networks
        self.soft_update(self.actor_local, self.actor_target, TAU)
        self.soft_update(self.critic_local, self.critic_target, TAU)
        
        
    def soft_update(self, local, target, tau):
        """
        Soft update
        
        𝜃_target = 𝜏*𝜃_local + (1 - 𝜏)*𝜃_target
        
        """
        # .parameters() is generator
        for target_param, local_param in zip(target.parameters(), local.parameters()):
            target_param.data.copy_(tau*local_param + (1-tau)*local_param)

## 2.Plot of Rewards
---
### 1.Training Code

In [8]:
from unityagents import UnityEnvironment
# select this option to load version 1 (with a single agent) of the environment
env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [9]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

# shape of its elements
print('Shape of next state: {}'.format(env_info.vector_observations.shape))
print('Shape of the rewards : {}'.format(env_info.rewards))
print('Shape of dones : {}'.format(env_info.local_done))

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.75471878e+00  -1.00000000e+00
   5.55726671e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
  -1.68164849e-01]
Shape of next state: (1, 33)
Shape of the rewards : [0.0]
Shape of dones : [False]


In [10]:
agent = DDPGAgent(state_size=state_size, action_size=action_size, seed=0)

In [14]:
def ddpg(n_episodes=1000, timestep_max=2000, print_every=100):
    scores = []
    score_deque = deque(maxlen=print_every)
    for i_episode in range(n_episodes):
        # Get Initial State
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        # noise reset
        agent.reset()
        # Episode score
        score = 0
        for t in range(timestep_max):
            action = agent.act(state)
            
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]
            done = env_info.local_done[0]
        
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break
        score_deque.append(score)
        scores.append(score)
        print('Episode: {}\t Average score: {}'.format(i_episode, np.mean(score_deque)))
        # save model parameters
        torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
        torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
        if i_episode%print_every == 0:
            print('Episode: {}\t Average score: {}'.format(i_episode, np.mean(score_deque)))
            
    return scores 

### 2. Results

In [15]:
import matplotlib.pyplot as plt

In [16]:
socres = ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores+1), 1), socres)
plt.ylabel('average score')
plt.xlabel('Episode')
plt.show()

KeyboardInterrupt: 

In [ ]:
env.close()

## 3. Ideas for Future Work

1. Hyper paramter를 조정해본다
    
    Authors get the optiaml hyper paratmeter by extensive trial to the task. and obviously the task in the paper and this project task are different. so there might be a better set of hyperparamters.
    
    